In [ ]:
sudo docker run -dit --name nvidia-10.1-container --gpus all nvidia/cuda:10.1-devel-ubuntu18.04 bash

In [ ]:
nvcc --compiler-options -Wall Rcf.cu -o Rcf.out

In [ ]:
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum ) {
double x=0.0;

if(threadIdx.x<=n-1){
x=a+(threadIdx.x+1/2.0)*h_hat;
data[threadIdx.x]=std::exp(-std::pow(x,2));
}
    *sum = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
  //*sum = thrust::reduce(thrust::seq, data , data + n, (double)0, thrust::plus<double>());
}


int main(int argc, char *argv[]){
    double suma=0.0;
    double *d_data;
    double *d_suma;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1e3; //número de subintervalos
    double objetivo=0.7468241328124271;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_suma,sizeof(double));
    h_hat=(b-a)/n;
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_suma); //1 bloque de n threads
    cudaDeviceSynchronize();
    cudaMemcpy(&suma, d_suma, sizeof(double), cudaMemcpyDeviceToHost);
    suma=h_hat*suma;
    cudaFree(d_data) ;
    cudaFree(d_suma) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,suma);
    printf("Error relativo de la solución: %1.15e\n", fabs(suma-objetivo)/fabs(objetivo));
    return 0;
}

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01

Error relativo de la solución: 4.104931878976858e-08

https://docs.nvidia.com/cuda/cutensor/api/cutensor.html#reduction-operations
https://github.com/NVIDIA/CUDALibrarySamples/tree/master/cuTENSOR

https://devblogs.nvidia.com/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/

https://stackoverflow.com/questions/14760893/using-cuda-math-functions-in-a-global-function-nsight-eclipse-edition

https://thrust.github.io/doc/group__reductions_ga6ac0fe1561f58692e85112bd1145ddff.html

https://stackoverflow.com/questions/20324277/reduction-in-cuda
https://github.com/thrust/thrust/issues/949
https://stackoverflow.com/questions/5510715/thrust-inside-user-written-kernels

http://thrust.github.io/

https://devtalk.nvidia.com/default/topic/1048650/cuda-programming-and-performance/question-about-thrust-library-with-kernel/

https://docs.nvidia.com/cuda/thrust/index.html


https://github.com/thrust/thrust/wiki/Quick-Start-Guide

https://github.com/thrust/thrust

In [ ]:
https://github.com/moderngpu/moderngpu/wiki

http://nvlabs.github.io/cub/#sec6

https://nvlabs.github.io/cub/structcub_1_1_device_reduce.html
http://nvlabs.github.io/cub/index.html
https://github.com/NVlabs/cub

https://stackoverflow.com/questions/42525713/cuda-how-to-sum-all-elements-of-an-array-into-one-number-within-the-gpu
http://nvlabs.github.io/cub/classcub_1_1_block_reduce.html

https://nvlabs.github.io/cub/structcub_1_1_device_reduce.html#aa4adabeb841b852a7a5ecf4f99a2daeb

https://devtalk.nvidia.com/default/topic/1062732/gpu-accelerated-libraries/cub-reduction-with-complex-number-and-multiplication/

In [ ]:
sudo docker run -dit --name nvidia-10.1-container --gpus all nvidia/cuda:10.1-devel-ubuntu18.04 bash


sudo docker exec -it nvidia-10.1-container bash

apt-get install wget unzip nano

wget https://github.com/NVlabs/cub/archive/1.8.0.zip

unzip 1.8.0.zip


nvcc --compiler-options -Wall suma.cu -o suma.out -I cub-1.8.0/cub/

In [ ]:
nvcc --compiler-options -Wall suma.cu -o suma.out -I cub-1.8.0/cub/


#include<stdio.h>
#include "cub-1.8.0/cub/cub.cuh"
void suma(int a, int b, int *c){
  void *d_temp_storage = NULL;
  size_t temp_storage_bytes = 0;
  int *d_in;
  int num_items=2;
  int i;

cudaMallocManaged((void **)&d_in,sizeof(int)*num_items);
for(i=0;i<num_items;i++)
 d_in[i] = i+5;

cub::DeviceReduce::Sum(d_temp_storage, temp_storage_bytes, d_in, c, num_items);
cudaMalloc(&d_temp_storage, temp_storage_bytes);
cub::DeviceReduce::Sum(d_temp_storage, temp_storage_bytes, d_in, c, num_items);
cudaFree(&d_temp_storage);
cudaFree(d_in);

}

int main(void){
	int c=0;
	int *device_c;
	cudaMalloc((void **)&device_c,sizeof(int));
        cudaSetDeviceFlags(cudaDeviceScheduleBlockingSync);
	suma(2,7,device_c);
	cudaMemcpy(&c, device_c, sizeof(int), cudaMemcpyDeviceToHost);
	printf("2+7 = %d\n", c);
	cudaFree(device_c);
	return 0;
}



https://stackoverflow.com/questions/25353749/getting-cub-devicescan-to-work-when-called-from-a-kernel

In [ ]:
Maybe see:

https://stackoverflow.com/questions/25353749/getting-cub-devicescan-to-work-when-called-from-a-kernel

#include<stdio.h>
#include "cub-1.8.0/cub/cub.cuh"


__global__ void suma(int a, int b, int *c){
  void *d_temp_storage = NULL;
  size_t temp_storage_bytes = 0;
  int *d_in;
  int num_items=2;
  int i;

cudaMallocManaged((void **)&d_in,sizeof(int)*num_items);
for(i=0;i<num_items;i++)
 d_in[i] = i+1;

cub::DeviceReduce::Sum(d_temp_storage, temp_storage_bytes, d_in, c, num_items);
cudaMalloc(&d_temp_storage, temp_storage_bytes);
cub::DeviceReduce::Sum(d_temp_storage, temp_storage_bytes, d_in, c, num_items);
cudaFree(&d_temp_storage);
cudaFree(d_in);

}

int main(void){
	int c=0;
	int *device_c;
	cudaMalloc((void **)&device_c,sizeof(int));
        cudaSetDeviceFlags(cudaDeviceScheduleBlockingSync);
	suma<<<1,1>>>(2,7,device_c);
	cudaMemcpy(&c, device_c, sizeof(int), cudaMemcpyDeviceToHost);
	printf("2+7 = %d\n", c);
	cudaFree(device_c);
	return 0;
}